In [1]:
import torch
import math
import numpy as np
from torch.profiler import profile, record_function, ProfilerActivity
path_train = '../data/train_data.pkl'
path_val = '../data/val_data.pkl'

noisy_imgs_1, noisy_imgs_2 = torch.load(path_train)
noisy_imgs_1 = noisy_imgs_1[0:1].cpu().float()/256
noisy_imgs_2 = noisy_imgs_2[0:1].cpu().float()/256

noisy_imgs , clean_imgs = torch.load(path_val)
torch.cuda.empty_cache()


In [2]:
import torch.nn as nn
import matplotlib.pyplot as plt

In [3]:
from torch import nn
from tqdm import tqdm
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


from PIL import Image
import torch 
import torchvision 
import torchvision.transforms as T 
from PIL import Image

transform = T.ToPILImage()
# model = UNet(m=10).to(device)
model = nn.Sequential(
    nn.Linear(32,100),
    nn.ReLU(),
    nn.Linear(100, 32)
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
mse = nn.MSELoss()


batch_size, nb_epochs = 50, 2

input = noisy_imgs_1.to(device).type(torch.float)
target = noisy_imgs_2.to(device).type(torch.float)

valid_inp = noisy_imgs.to(device).float()/256
valid_target = clean_imgs.to(device).float()/256

In [4]:
loss_tot = torch.zeros(nb_epochs)
loss_valid = torch.zeros(nb_epochs)
print(loss_tot)
for e in tqdm(range(nb_epochs)):
    model.train()
    for i in range(len(input)):
        with torch.autograd.graph.saved_tensors_hooks(pack_hook, unpack_hook):
            output = model(input[i])
            loss = mse(output, target[i])
            loss_tot[e] += loss.item()    
            optimizer.zero_grad()
            loss.backward(retain_graph=True)
        optimizer.step()
    
    model.eval()
    valid_output = model(valid_inp)
    loss_valid[e] = mse(valid_output, valid_target).item()

    if torch.remainder(torch.tensor(e), 100)==0:
        plt.figure()
        im = 0
        im2 = 0
        plt.subplot(2, 3, 1)
        plt.imshow(transform(input[i][im]))
        plt.colorbar()
        plt.subplot(2, 3, 2)
        plt.imshow(transform(output[im]))
        plt.colorbar()
        plt.subplot(2, 3, 3)
        plt.imshow(transform(target[i][im]))
        plt.colorbar()
        plt.subplot(2, 3, 4)
        plt.imshow(transform(valid_inp[im2]))
        plt.colorbar()
        plt.subplot(2, 3, 5)
        plt.imshow(transform(valid_output[im2]))
        plt.colorbar()
        plt.subplot(2, 3, 6)
        plt.imshow(transform(valid_target[im2]))
        plt.colorbar()
        plt.show()

  0%|          | 0/2 [00:00<?, ?it/s]

tensor([0., 0.])


NameError: name 'pack_hook' is not defined

In [5]:
loss.grad_fn


NameError: name 'loss' is not defined

In [6]:
def pack_hook(x):
    print("Packing", x.grad_fn)
    return x

def unpack_hook(x):
    print("Unpacking", x.grad_fn)
    return x

a = torch.ones(5, requires_grad=True) 
b = torch.ones(5, requires_grad=True) * 2
with torch.autograd.graph.saved_tensors_hooks(pack_hook, unpack_hook):
    y = a * b

y.sum().backward()
for param in model.parameters():
    print(param.grad)

Packing None
Packing <MulBackward0 object at 0x7ff4055cbcd0>
Unpacking None
Unpacking <MulBackward0 object at 0x7ff4055cb670>
None
None
None
None


In [7]:
for param in model.parameters():
    print(param)
    print(param.grad)

Parameter containing:
tensor([[-0.1395,  0.1044,  0.0699,  ..., -0.0688,  0.1290, -0.0136],
        [ 0.1739,  0.0215,  0.1039,  ..., -0.0597, -0.1578, -0.0652],
        [ 0.1395,  0.1026, -0.0664,  ..., -0.1323, -0.0120,  0.1674],
        ...,
        [ 0.1614, -0.1582,  0.1572,  ...,  0.1293, -0.0373, -0.1709],
        [-0.1542, -0.1628,  0.0750,  ..., -0.0703, -0.0104,  0.1745],
        [ 0.1358, -0.1097,  0.1646,  ..., -0.1431,  0.0641,  0.1767]],
       requires_grad=True)
None
Parameter containing:
tensor([-0.0865, -0.1354,  0.1431, -0.0722, -0.0232, -0.0983,  0.0983, -0.0306,
        -0.0906,  0.1284, -0.0844, -0.0858,  0.1458,  0.1527,  0.0432,  0.0154,
        -0.1741,  0.1381, -0.0223,  0.0559,  0.1607,  0.0329, -0.0994, -0.0819,
         0.1051,  0.0989,  0.0545,  0.0862, -0.0085, -0.0838, -0.0527,  0.0626,
         0.0955,  0.0160,  0.1733,  0.1276, -0.0569,  0.1512, -0.1199, -0.1473,
         0.0900,  0.0088, -0.0015, -0.0227, -0.0821,  0.0275, -0.0999,  0.1200,
        -0

In [8]:
from torch.nn.functional import unfold
class relu(object) :
    def __init__(self):
        pass
    def forward(self, input) :
        self.input = input
        self.positif_mask = (input > 0)
        return self.positif_mask*(input)
    def backward(self, gradwrtoutput) :
        self.input.grad = self.positif_mask.int()*gradwrtoutput
        print('Our backward: ', self.input.grad)
    def param(self) :
        return []

class sigmoid(object) :
    def forward(self, input) :
        self.input = input
        self.output = 1/(1 + math.e**(-input))
        return  self.output
    def backward(self, gradwrtoutput ) :
        self.input.grad = self.output * (1-self.output) * gradwrtoutput
    def param(self) :
        return []

class convolution(object):
    def __init__(self, inp_channels, out_channels, kernel_size = (3,3), stride = (1,1), padding = (0,0), dilation = (1,1)):
        self.inp_channels = inp_channels
        self.out_channels = out_channels
        self.kernel = torch.empty((out_channels, kernel_size[0], kernel_size[1])).normal_()
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        
    def forward(self, input):
        self.batch_size = input.size(0)
        self.input = input
        up = (input.size(-2) + 2 * self.padding[0] - self.dilation[0] * (self.kernel_size[0] - 1) - 1)
        down = self.stride[0]
        h = torch.tensor(up/down + 1).floor().int()
        up = (input.size(-1) + 2 * self.padding[1] - self.dilation[1] * (self.kernel_size[1] - 1) - 1)
        down = self.stride[1]
        w = torch.tensor(up/down + 1).floor().int()
        unf = unfold(input, kernel_size=self.kernel_size)
        conv = self.kernel.view(self.out_channels, -1) @ unf
#         print(conv.view(1, 1, 2, 2))
#         print(h, w)
        return conv.view(self.batch_size, self.out_channels, h, w)

    def backward(self, gradwrtoutput):
        flattened = gradwrtoutput.view(self.batch_size,self.out_channels,-1)
        print(flattened)
        pass
    def param(self) :
        return [self.kernel]
    
class mse(object):
    def forward(self, input, target):
        self.input = input
        self.target = target
        return (input - target).pow(2).mean(dim=[1, 2, 3]).sum()
    def backward(self, gradwrtoutput):
        self.input.grad = 2*(self.input-self.target)/(self.input.size(-3)*self.input.size(-2)*self.input.size(-1))
        
        


In [16]:
MSE = nn.MSELoss()

target = torch.randn(1, 1, 3, 3)
print(target)

input = torch.randn(1, 1, 3, 3, requires_grad=True)
input2 = input.detach().requires_grad_(True)

print(input)
# print(input2)
model = sigmoid()
model2 = torch.nn.Sigmoid()

out = model.forward(input)
out2 = model2.forward(input2)

print(out)
print(out2)
my_mse = mse()
loss = my_mse.forward(out, target)
loss2 = MSE(out2, target)

print(loss)
print(loss2)
my_mse.backward(loss)
model.backward(my_mse.input.grad)

loss2.backward()

# print(model.input.grad)

print(input.grad)
print(input2.grad)


tensor([[[[-0.0440,  0.9203,  0.1944],
          [-0.5946, -0.4954, -1.9285],
          [-1.0043, -1.0796, -0.2933]]]])
tensor([[[[ 1.0904,  0.8289, -1.1416],
          [ 1.1780,  0.8426, -1.1685],
          [ 0.5877, -0.2360,  0.0489]]]], requires_grad=True)
tensor([[[[0.7485, 0.6961, 0.2420],
          [0.7646, 0.6990, 0.2371],
          [0.6428, 0.4413, 0.5122]]]], grad_fn=<MulBackward0>)
tensor([[[[0.7485, 0.6961, 0.2420],
          [0.7646, 0.6990, 0.2371],
          [0.6428, 0.4413, 0.5122]]]], grad_fn=<SigmoidBackward0>)
tensor(1.5910, grad_fn=<SumBackward0>)
tensor(1.5910, grad_fn=<MseLossBackward0>)
tensor([[[[ 0.0332, -0.0105,  0.0019],
          [ 0.0544,  0.0558,  0.0871],
          [ 0.0840,  0.0833,  0.0447]]]], grad_fn=<MulBackward0>)
tensor([[[[ 0.0332, -0.0105,  0.0019],
          [ 0.0544,  0.0558,  0.0871],
          [ 0.0840,  0.0833,  0.0447]]]])


In [17]:
test_conv = convolution(1, 3, kernel_size = (3,3))
dummy_input = (torch.empty((1,1,7,7)).normal_()*10)
test_conv.backward(test_conv.forward(dummy_input))

tensor([[[ 1.4373e+01,  8.9594e+00,  8.2914e+00, -3.2300e+01, -8.2132e+00,
          -3.5403e+00, -3.8366e+01,  4.4688e+01, -4.9594e+01,  6.8135e+00,
          -2.9135e+01, -6.3563e+00,  1.9680e+01,  1.0055e+01, -4.6699e+00,
           1.2237e+01,  7.4446e-01, -2.1380e+01,  2.1600e+00,  8.1567e+00,
          -9.0960e+00,  4.4997e+01, -2.2331e+01,  4.7453e+01,  1.4016e+00],
         [-2.1148e+01, -4.0760e+01,  1.7178e+01,  5.6863e+01, -6.8904e+00,
          -6.7906e-02, -3.9611e+01,  2.9627e+00,  4.0970e+01, -1.6927e+01,
           3.9700e+01, -1.3491e+01,  2.9640e+00, -7.9330e+01,  3.3416e+01,
          -5.3917e+00,  2.1418e+01,  4.4497e+01, -7.2982e+01,  4.7925e+01,
          -5.2293e+01,  6.1290e+00,  1.6297e+01,  3.4601e+00,  1.1976e+01],
         [ 3.2122e+00,  4.1303e+00,  1.8777e+01,  8.4691e+00, -8.9788e+00,
           1.0548e+01, -3.5732e+00,  2.5001e+01, -3.3122e+01, -6.3945e+00,
           4.9866e+00, -6.2900e+00,  9.1395e+00, -4.7929e+01, -2.1028e+01,
          -3.8644e+01, 

In [12]:
inp = torch.normal(torch.zeros(5), torch.ones(5)*2)
print('input: ', inp)
tar = torch.round(inp).requires_grad_(True)
model= relu()
model_comp = torch.nn.ReLU()
inp
inp.requires_grad_(True)
inp_comp = inp.clone()
out = model.forward(inp)
out_comp = model_comp(inp)
print('output: ', out)
print('output_comp: ', out_comp)

mse_comp = torch.nn.MSELoss()
loss = mse(out,tar)
loss_comp = mse_comp(out_comp,tar)
print('loss: ', loss)
print('loss_comp: ', loss_comp)


print(model.backward(loss))
loss_comp.backward()
print(inp_comp.grad)
print(out_comp.grad)
# torch.autograd.grad(out)


input:  tensor([-1.1305,  1.1211,  2.8137,  3.1344,  0.6033])
output:  tensor([-0.0000, 1.1211, 2.8137, 3.1344, 0.6033], grad_fn=<MulBackward0>)
output_comp:  tensor([0.0000, 1.1211, 2.8137, 3.1344, 0.6033], grad_fn=<ReluBackward0>)


TypeError: mse() takes no arguments

In [ ]:
import matplotlib.pyplot as plt
plt.plot(inp,out, '.')
plt.plot(inp,tar, '.')
plt.plot(inp, loss, '.')

In [ ]:
(a > 0).int()